# Problem Description

Given two squares on a two dimensional plane, find a line that would cut these two squares in half. Assume that the top and bottom sides of the square run parallel to the x-axis.

# Solution

The solution provided involves creating classes to represent points and lines, and methods to identify the best line that passes through the maximum number of points.

### GraphPoint Class

Represents a point on the graph with x and y coordinates.

In [9]:
class GraphPoint:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

### Line Class

Represents a line defined by two points. It calculates the slope and y-intercept of the line and includes helper methods to determine if two lines are equivalent within a small epsilon value.

In [10]:
class Line:
    def __init__(self, start: GraphPoint, end: GraphPoint) -> None:
        self.epsilon: float = 0.0001
        self.slope: float = 0
        self.yintercept: float = 0
        self.infinite_slope: bool = False
        self.start: GraphPoint = start
        self.end: GraphPoint = end
        if abs(start.x - end.x) < self.epsilon:
            self.infinite_slope = True
            self.yintercept = start.x
        else:
            self.slope = (end.y - start.y) / (end.x - start.x)
            self.yintercept = end.y - self.slope * end.x

    def FloorToNearestEpsilon(self, d: float) -> float:
        r: int = int(d // self.epsilon)
        return float(r * self.epsilon)

    def IsEquivalentHelper(self, a: float, b: float) -> bool:
        return abs(a - b) < self.epsilon

    def IsEquivalent(self, a: 'Line') -> bool:
        if self.IsEquivalentHelper(a.slope, self.slope) and self.IsEquivalentHelper(a.yintercept, self.yintercept) and a.infinite_slope == self.infinite_slope:
            return True
        return False

### BestLine Class

Contains methods to find the best line that passes through the most points.
   * FindBestLine: Finds the best line that passes through the most points.
   * GetListOfLines: Generates a list of lines from all pairs of points.
   * GetBestLine: Finds the best line by counting equivalent lines.
   * CountEquivalentLines: Counts lines that are equivalent to a given line.
   * CountEquivalentLinesHelper: Helper method to count lines within a list that are equivalent to a given line.

In [11]:
class BestLine:

    def FindBestLine(self, points: list[GraphPoint]) -> Line:
        lineBySlope: dict[float, Line] = self.GetListOfLines(points)
        return self.GetBestLine(lineBySlope)

    def GetListOfLines(self, points: list[GraphPoint]) -> dict[float, Line]:
        '''
        Add each pair of points as a line to the list
        '''
        linesBySlope: dict[float, Line] = {}
        for i in range(len(points)):
            for j in range(i + 1, len(points), 1):
                line: Line = Line(points[i], points[j])
                key: float = line.FloorToNearestEpsilon(line.slope)
                if key not in linesBySlope:
                    linesBySlope[key] = []
                linesBySlope[key].append(line)
        return linesBySlope

    def GetBestLine(self, lineBySlope: dict[float, Line]) -> Line:
        bestLine: Line = None
        bestCount: int = 0
        slopes: list[float] = list(lineBySlope.keys())
        for slope in slopes:
            lines: list[Line] = lineBySlope[slope]
            for line in lines:
                # count lines that are equivalent to current line
                count: int = self.CountEquivalentLines(lineBySlope, line)
                # if better than current line, replace it
                if count > bestCount:
                    bestLine = line
                    bestCount = count
                    print(f"Best line: Slope={bestLine.slope}, Y-intercept={bestLine.yintercept}, Count={bestCount}")
        return bestLine

    def CountEquivalentLines(self, lineBySlope: dict[float, Line], line: Line) -> int:
        '''
        Check hashmap for lines that are equivalent. Note that we need to check one
        epsilon above and below the actual slope since we're defining two lines as
        equivalent if they're within an epsilon of each other
        '''
        key: float = line.FloorToNearestEpsilon(line.slope)
        count: int = self.CountEquivalentLinesHelper(lineBySlope.get(key, []), line)
        count += self.CountEquivalentLinesHelper(lineBySlope.get(key - line.epsilon, []), line)
        count += self.CountEquivalentLinesHelper(lineBySlope.get(key + line.epsilon, []), line)
        return count

    def CountEquivalentLinesHelper(self, lines: list[Line], line: Line) -> int:
        '''
        Count lines within an array of lines which are equivalent (slope and
        y-intercept are within an epsilon value) to a given line
        '''
        if lines is None:
            return 0
        count: int = 0
        for parallelLine in lines:
            if parallelLine.IsEquivalent(line):
                count += 1
        return count

## Example Usage

Here's how you might use these classes to find the best line that passes through the most points:

In [12]:
# Example points
points = [
    GraphPoint(1, 2),
    GraphPoint(2, 3),
    GraphPoint(3, 4),
    GraphPoint(4, 5),
    GraphPoint(6, 8)
]

solution = BestLine()
best_line = solution.FindBestLine(points)

print(f"Best line passes through points: ({best_line.start.x}, {best_line.start.y}) and ({best_line.end.x}, {best_line.end.y})")

Best line: Slope=1.0, Y-intercept=1.0, Count=6
Best line passes through points: (1, 2) and (2, 3)


# Literature

The contents base on the following literature:

* Gayle Laakmann McDowell, *Cracking the Coding Interview*, [Link](https://www.crackingthecodinginterview.com/).

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).